# Assignment 4

## 1.Using the same Dmart file, calculate the weekly returns for the entire year 

2.Populate a table of returns with week number of month(1,2,3,4,5) and month name        

Please note:
1.Week starts from Monday
2. Use while loop to loop through the dataframe and not any form of groupby, directly or indirectly
3. You buy at the open of the first day of the week and sell at the close of the last day of week
4. If the end of month falls on Wednesday, you exit your position on Wednesday close. New week of the new month starts from Thursday


In [1]:
import pandas as pd
import numpy as np
import calendar
import datetime
from dateutil import parser

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Please change the path of the file as per your pc after r'
path = r'C:\Users\admin\Desktop\Sakshi\6. Python\Assignment 4\Dmart.csv'

##### Dmart file = df

In [3]:
df = pd.read_csv(path)

In [4]:
#Step 1 -> converted date/time column to date/time format

df['Date'] = np.nan

for i in range (0,len(df['Date/Time'])):
    date_string = df['Date/Time'][i]
    date_object = parser.parse(date_string)
    df['Date'][i] = date_object
    
#display(df)

In [5]:
#Adding a day column, month column, week number column
df['Day'] = np.nan
df['Month'] = np.nan
df['Week Number'] = np.nan


for i in range (0,len(df['Date/Time'])):
    df['Day'][i] = (df['Date'][i]).strftime("%A")
    df['Month'][i] = (df['Date'][i]).strftime("%m")
    df['Week Number'][i] = (df['Date'][i]).strftime("%W")  
    
#display(df)    

In [6]:
#adding a column to check if day is wednesday or no

df['Checking day is wednesday or no'] = df['Day'].apply (lambda x: '1' if x == 'Wednesday' else '0')

#display(df)

In [7]:
#adjusting month to another cell
i = 0
while i < len(df['Date/Time']):
    for i in range (0,len(df['Date/Time'])):
        if i == 251:
            df.loc[i,'NewLocation'] = df.loc[i,'Month']
        else:
            df.loc[i,'NewLocation'] = df.loc[i+1,'Month']
        i += 1
#display(df)

In [8]:
#finding month end
i = 0
while i < len(df['Date/Time']):
    for i in range (0,len(df['Date/Time'])):
        if df.loc[i,'Month'] == df.loc[i,'NewLocation']:
            df.loc[i,'MonthCheck'] = '0'
        else:
            df.loc[i,'MonthCheck'] = '1'
        i += 1
    
#display(df)

In [9]:
# combining month end and wednesday close
i = 0
while i < len(df['Date/Time']):
    for i in range (0,len(df['Date/Time'])):
        if ((df.loc[i,'MonthCheck'] == '1') and (df.loc[i,'Checking day is wednesday or no'] == '1')):
            df.loc[i,'WednesdayClose'] = '1'
        else:
            df.loc[i,'WednesdayClose'] = '0'
        i += 1
    
#display(df)    

In [10]:
#close counter
i = 0
while i < len(df['Date/Time']):
    for i in range (0,len(df['Date/Time'])):
        if i == 0:
            df.loc[i,'CloseApprox'] = '0'
        elif i == 251:
            df.loc[i,'CloseApprox'] = '1'
        elif (df.loc[i,'Week Number']) == (df.loc[i+1,'Week Number']):
            df.loc[i,'CloseApprox'] = '0'
        else:
            df.loc[i,'CloseApprox'] = '1'
        i += 1    
        
#display(df)

In [11]:
#to determine the close
i = 0
while i < len(df['Date/Time']):
    for i in range (0,len(df['Date/Time'])):
        if df.loc[i,'WednesdayClose'] == '1':
            df.loc[i,'CloseCount'] = 1
        elif df.loc[i,'CloseApprox'] == '1':
            df.loc[i,'CloseCount'] = 1
        else:
            df.loc[i,'CloseCount'] = 0
        i += 1
#display(df)

In [12]:
i = 0
while i < len(df['Date/Time']):
    for i in range (0,len(df['Date/Time'])):
        if i == 0:
            df.loc[i,'OpenCount'] = 1
        elif i == 251:
            df.loc[i,'OpenCount'] = 0
        elif df.loc[i,'Week Number'] != df.loc[i-1,'Week Number']:
            df.loc[i,'OpenCount'] = 1
        else:
            df.loc[i,'OpenCount'] = 0
        i += 1
#display(df)

In [13]:
#counters of close and open
i = 0
summationclose = 0
for i in range(0,len(df['Date/Time'])):
    summationclose += df.loc[i,'CloseCount']
    df.loc[i,'CloseCounter'] = summationclose
    
j = 0
summationopen = 0
for j in range (0,len(df['Date/Time'])):
    summationopen += df.loc[j,'OpenCount']
    df.loc[j,'OpenCounter'] = summationopen
        
#display(df)
        

In [14]:
#close counter helper
for i in range(0,len(df['Date/Time'])):
    if i == 0:
        df.loc[i,'CloseCounterHelper'] = df.loc[i,'CloseCounter'] 
    elif df.loc[i,'CloseCounter'] != df.loc[i,'OpenCounter']:
        df.loc[i,'CloseCounterHelper'] = df.loc[i-1,'CloseCounter']
    else:
        df.loc[i,'CloseCounterHelper'] = df.loc[i,'CloseCounter']
#display(df)

In [19]:
i = 0
while i < len(df):
    for i in range (0,len(df['Date/Time'])):
        if i == 0:
            df.loc[i,'OpenCounterFinal'] = df.loc[i,'OpenCounter']
        elif df.loc[i,'OpenCounter'] != df.loc[i-1,'OpenCounter']:
            df.loc[i,'OpenCounterFinal'] = df.loc[i,'OpenCounter']
        else:
            df.loc[i,'OpenCounterFinal'] = 0
        i+= 1
        
        

#display(df)

In [22]:
#i = 0
#while i < len(df):
for i in range (0,len(df['Date/Time'])):
    if i == 0:
        df.loc[i,'CloseCounterFinal'] = df.loc[i,'CloseCounterHelper']
    elif i == 251:
        df.loc[i,'CloseCounterFinal'] = df.loc[i,'CloseCounterHelper']
    elif (df.loc[i,'CloseCounterHelper'] == df.loc[i,'OpenCounterFinal']):
        #if i == 251:
            #df.loc[i,'CloseCounterFinal'] = df.loc[i,'CloseCounterHelper']
        #if (df.loc[i,'CloseCounterHelper'] != df.loc[i+1,'CloseCounterHelper']):
            #df.loc[i,'CloseCounterFinal'] = df.loc[i,'CloseCounterHelper']
        #elif (df.loc[i,'CloseCounterHelper'] != df.loc[i-1,'CloseCounterHelper']):
            #df.loc[i,'CloseCounterFinal'] = 0 #df.loc[i,'CloseCounterHelper']            
        #else:
        df.loc[i,'CloseCounterFinal'] = 0
    elif (df.loc[i,'CloseCounterHelper'] != df.loc[i-1,'CloseCounterHelper']):
        df.loc[i,'CloseCounterFinal'] = df.loc[i,'CloseCounterHelper']
    elif (df.loc[i,'CloseCounterHelper'] != df.loc[i+1,'CloseCounterHelper']):
        df.loc[i,'CloseCounterFinal'] = df.loc[i,'CloseCounterHelper']
    else:
        df.loc[i,'CloseCounterFinal'] = 0
        #i += 1
display(df)



,Ticker,Date/Time,Open,Low,High,Close,Date,Day,Month,Week Number,...,MonthCheck,WednesdayClose,CloseApprox,CloseCount,OpenCount,CloseCounter,OpenCounter,CloseCounterHelper,OpenCounterFinal,CloseCounterFinal
0,DMART.EQ-NSE,1/1/20,1849.70,1820.00,1867.00,1828.50,2020-01-01 00:00:00,Wednesday,01,00,...,0,0,0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,DMART.EQ-NSE,1/2/20,1831.00,1815.00,1854.85,1824.20,2020-01-02 00:00:00,Thursday,01,00,...,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,DMART.EQ-NSE,1/3/20,1827.00,1827.00,1853.00,1843.55,2020-01-03 00:00:00,Friday,01,00,...,0,0,1,1.0,0.0,1.0,1.0,1.0,0.0,1.0
3,DMART.EQ-NSE,1/6/20,1827.00,1780.00,1829.00,1789.90,2020-01-06 00:00:00,Monday,01,01,...,0,0,0,0.0,1.0,1.0,2.0,1.0,2.0,0.0
4,DMART.EQ-NSE,1/7/20,1805.00,1805.00,1837.50,1815.75,2020-01-07 00:00:00,Tuesday,01,01,...,0,0,0,0.0,0.0,1.0,2.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,DMART.EQ-NSE,12/24/2020,2661.85,2604.80,2700.00,2673.10,2020-12-24 00:00:00,Thursday,12,51,...,0,0,1,1.0,0.0,53.0,52.0,52.0,0.0,52.0
248,DMART.EQ-NSE,12/28/2020,2685.95,2655.00,2698.95,2678.00,2020-12-28 00:00:00,Monday,12,52,...,0,0,0,0.0,1.0,53.0,53.0,53.0,53.0,0.0
249,DMART.EQ-NSE,12/29/2020,2690.00,2645.00,2694.70,2665.70,2020-12-29 00:00:00,Tuesday,12,52,...,0,0,0,0.0,0.0,53.0,53.0,53.0,0.0,0.0
250,DMART.EQ-NSE,12/30/2020,2666.20,2642.35,2725.00,2680.30,2020-12-30 00:00:00,Wednesday,12,52,...,0,0,0,0.0,0.0,53.0,53.0,53.0,0.0,0.0


In [23]:
df.to_csv('DmartOutput.csv')